# Discussion 8

## Databases

A _relational database_ is a collection of tables. Usually, each table is connected to one or more other tables through a common column.

_Structured query language_ (SQL) is a language designed for accessing information in relational databases.

A free SQL tutorial is available [here](https://www.w3schools.com/sql/).

### `sqlite3`

SQLite is possibly [the most-used database engine in the world](https://sqlite.org/mostdeployed.html). SQLite's popularity is partly due to its reliability, easy setup, and broad range of features.

Python has built-in support for SQLite databases through the `sqlite3` module.

In [3]:
import sqlite3 as sql

To connect to a database, use the module's `connect()` function. This is similar to opening a file; you should close the database when you're done using it.

In [4]:
db = sql.connect("suppliers.sqlite")

To execute a SQL command, use the connection's `.execute()` method. This returns a _cursor_, which is a pointer to the results in the database (imagine a finger pointing at the results).

SQLite databases store metadata in a special table called `sqlite_master`. We can use `sqlite_master` to find out the names of the other tables in the database.

In [4]:
cursor = db.execute("SELECT * FROM sqlite_master")

To get the results from the database, use one of the cursor's fetch methods. The `.fetchall()` method returns all rows in the result.

In [5]:
cursor.fetchall()

[(u'table',
  u'Suppliers',
  u'Suppliers',
  2,
  u'CREATE TABLE Suppliers (\nSupplierID integer,\nSupplierName text,\nStatus integer,\nCity text,\nPRIMARY KEY(SupplierID)\n)'),
 (u'table',
  u'Parts',
  u'Parts',
  3,
  u'CREATE TABLE Parts (\nPartID integer,\nPartName text,\nColor text,\nWeight real,\nCity text,\nPRIMARY KEY(PartID)\n)'),
 (u'table',
  u'SupplierParts',
  u'SupplierParts',
  4,
  u'CREATE TABLE SupplierParts (\nPartID integer,\nSupplierID integer,\nQty integer,\nPRIMARY KEY(PartID, SupplierID)\n)'),
 (u'index', u'sqlite_autoindex_SupplierParts_1', u'SupplierParts', 5, None),
 (u'table',
  u'NewTable',
  u'NewTable',
  6,
  u'CREATE TABLE NewTable(\n  PartID INT,\n  PartName TEXT,\n  Color TEXT,\n  Weight REAL,\n  City TEXT\n)')]

By default, `sqlite3` will return rows as tuples. If you'd rather have the rows as dictionaries indexed by column name, set the `.row_factory` attribute on the database connection.

In [6]:
db.row_factory = sql.Row

Now the rows will behave like dictionaries:

In [7]:
cursor = db.execute("SELECT * FROM sqlite_master")
rows = cursor.fetchall()

In [9]:
rows[0].keys()

['type', 'name', 'tbl_name', 'rootpage', 'sql']

In [10]:
rows[0]['name']

u'Suppliers'

Don't forget to close the database when you're done!

In [ ]:
# db.close()

### `sqlalchemy`

The `sqlalchemy` package is a unified interface for a variety of different SQL database formats (more than just SQLite). See the [tutorial](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) for more details.

### Pandas

The easiest way to view database tables is to load them into a data frame with Pandas.

You still need to connect to the database first with `sqlite3` or `sqlalchemy`.

The `pd.read_sql()` function takes a SQL query and an open database connection as arguments. The result of the query is returned as a data frame.

In [5]:
import pandas as pd

pd.read_sql("SELECT * FROM sqlite_master", db)

,type,name,tbl_name,rootpage,sql
0,table,Suppliers,Suppliers,2,"CREATE TABLE Suppliers (\nSupplierID integer,\..."
1,table,Parts,Parts,3,"CREATE TABLE Parts (\nPartID integer,\nPartNam..."
2,table,SupplierParts,SupplierParts,4,"CREATE TABLE SupplierParts (\nPartID integer,\..."
3,index,sqlite_autoindex_SupplierParts_1,SupplierParts,5,None
4,table,NewTable,NewTable,6,"CREATE TABLE NewTable(\n PartID INT,\n PartN..."
5,table,NewTable2,NewTable2,7,"CREATE TABLE NewTable2(\n PartID INT,\n Part..."
6,table,Fresh,Fresh,8,"CREATE TABLE ""Fresh"" (\n""index"" INTEGER,\n ""f..."
7,index,ix_Fresh_index,Fresh,10,"CREATE INDEX ""ix_Fresh_index""ON ""Fresh"" (""index"")"


In [14]:
pd.read_sql("SELECT * FROM Suppliers", db)

,SupplierID,SupplierName,Status,City
0,1,Smith,20,London
1,2,Jones,10,Paris
2,3,Blake,30,Paris
3,4,Clark,20,London
4,5,Adams,30,Athens


Keep in mind that Pandas operations are slower and use more memory than SQL operations.

Joins and simple subsetting can almost always be done with SQL!

#### Taking Subsets: `LIMIT` and `WHERE`

In [15]:
pd.read_sql("SELECT * FROM SupplierParts LIMIT 3", db)

# Don't do this:
# pd.read_sql("SELECT * FROM SupplierParts", db).head()

,PartID,SupplierID,Qty
0,1,1,300
1,1,2,200
2,1,3,400


In SQL, `=` tests equality.

In [16]:
pd.read_sql("SELECT * FROM Suppliers WHERE Status = 20", db)

,SupplierID,SupplierName,Status,City
0,1,Smith,20,London
1,4,Clark,20,London


In [17]:
pd.read_sql("SELECT * FROM Suppliers WHERE City = 'Paris'", db)

,SupplierID,SupplierName,Status,City
0,2,Jones,10,Paris
1,3,Blake,30,Paris


You can use `AND` and `OR` to combine conditions. You can also use parenthesis to indicate the order of operations.

In [8]:
pd.read_sql("SELECT * FROM Suppliers WHERE Status = 10 AND City = 'Paris'", db)

,SupplierID,SupplierName,Status,City
0,2,Jones,10,Paris


SQL's `LIKE` keyword does simple pattern-matching language for strings. This is less powerful than regular expressions, but still useful.

* `%` matches zero or more of any character, similar to regex `.*`
* `-` matches any one character, similar to regex `.`
* `[]` matches any one of the characters you put inside the brackects, identical to regex `[]`

In [9]:
pd.read_sql("SELECT * FROM Suppliers WHERE SupplierName LIKE '%s'", db)

,SupplierID,SupplierName,Status,City
0,2,Jones,10,Paris
1,5,Adams,30,Athens


The `BETWEEN` keyword is useful for selecting ranges.

You can use `BETWEEN` with dates in this format:
```SQL
SELECT * FROM Table WHERE Date BETWEEN #01/15/2017# AND #02/16/2017#
```

In [10]:
pd.read_sql("SELECT * FROM Suppliers WHERE Status BETWEEN 20 AND 30", db)

,SupplierID,SupplierName,Status,City
0,1,Smith,20,London
1,3,Blake,30,Paris
2,4,Clark,20,London
3,5,Adams,30,Athens


#### Grouping and Aggregating: `GROUP BY`

In [24]:
pd.read_sql("SELECT * FROM Parts", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,5,Cam,Blue,12.0,Paris
5,6,Cog,Red,19.0,London


In [34]:
df = pd.read_sql("SELECT PartName, AVG(Weight) AS Avg_Weight, COUNT(*) AS Count FROM Parts GROUP BY PartName", db)

In [35]:
df

,PartName,Avg_Weight,Count
0,Bolt,17.0,1
1,Cam,12.0,1
2,Cog,19.0,1
3,Nut,12.0,1
4,Screw,15.5,2


To do subsetting and aggregation in one query, use `GROUP BY` with `HAVING`. The tutorial linked above has more details.

#### Joins: `INNER JOIN` and `LEFT JOIN`

Joins combine related information from two or more different tables.

In [38]:
pd.read_sql("SELECT * FROM Parts INNER JOIN SupplierParts ON Parts.PartID = SupplierParts.PartID", db)

,PartID,PartName,Color,Weight,City,PartID,SupplierID,Qty
0,1,Nut,Red,12.0,London,1,1,300
1,1,Nut,Red,12.0,London,1,2,200
2,1,Nut,Red,12.0,London,1,3,400
3,1,Nut,Red,12.0,London,1,4,200
4,1,Nut,Red,12.0,London,1,5,100
5,1,Nut,Red,12.0,London,1,6,100
6,2,Bolt,Green,17.0,Paris,2,1,300
7,2,Bolt,Green,17.0,Paris,2,2,400
8,3,Screw,Blue,17.0,Oslo,3,2,200
9,4,Screw,Red,14.0,London,4,2,200


In [40]:
pd.read_sql(
    "SELECT L.*, R.SupplierID, R.Qty "
    "FROM Parts AS L INNER JOIN SupplierParts AS R ON L.PartID = R.PartID", db
)

,PartID,PartName,Color,Weight,City,SupplierID,Qty
0,1,Nut,Red,12.0,London,1,300
1,1,Nut,Red,12.0,London,2,200
2,1,Nut,Red,12.0,London,3,400
3,1,Nut,Red,12.0,London,4,200
4,1,Nut,Red,12.0,London,5,100
5,1,Nut,Red,12.0,London,6,100
6,2,Bolt,Green,17.0,Paris,1,300
7,2,Bolt,Green,17.0,Paris,2,400
8,3,Screw,Blue,17.0,Oslo,2,200
9,4,Screw,Red,14.0,London,2,200


In [41]:
pd.read_sql(
    "SELECT L.*, R.SupplierID, R.Qty "
    "FROM Parts AS L LEFT JOIN SupplierParts AS R ON L.PartID = R.PartID", db
)

,PartID,PartName,Color,Weight,City,SupplierID,Qty
0,1,Nut,Red,12.0,London,1.0,300.0
1,1,Nut,Red,12.0,London,2.0,200.0
2,1,Nut,Red,12.0,London,3.0,400.0
3,1,Nut,Red,12.0,London,4.0,200.0
4,1,Nut,Red,12.0,London,5.0,100.0
5,1,Nut,Red,12.0,London,6.0,100.0
6,2,Bolt,Green,17.0,Paris,1.0,300.0
7,2,Bolt,Green,17.0,Paris,2.0,400.0
8,3,Screw,Blue,17.0,Oslo,2.0,200.0
9,4,Screw,Red,14.0,London,2.0,200.0


### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create - `CREATE`
* Read - `SELECT`
* Update - `UPDATE`
* Delete - `DROP`

For a data analysis, you'll usually only need to read data. The commands discussed below are most useful if you need to create or add data to database.

In [14]:
# Create a new table based on data already in the database.
# Use the connection rather than Pandas, since no table is returned.
db.execute("CREATE TABLE NewTable AS SELECT * FROM Parts WHERE Weight > 14")

In [15]:
pd.read_sql("SELECT * FROM NewTable", db)

,PartID,PartName,Color,Weight,City
0,2,Bolt,Green,17.0,Paris
1,3,Screw,Blue,17.0,Oslo
2,6,Cog,Red,19.0,London


In [16]:
# Delete the table we just created. Be careful--there is no undo!
db.execute("DROP TABLE NewTable")

In [ ]:
# Create a new table based on external data.
df = pd.read_csv("fresh.csv")

df.to_sql("Fresh", db)

You can create an entirely new database by connecting to the name you want the database to have. For instance:

In [ ]:
new_db = sql.connect("my_new_database.sqlite")